In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path
import torch
import torch.nn as nn
from torchvision import models

In [3]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [5]:
from src.portfolio_projects.constants import *
from src.portfolio_projects.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [6]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        # Load VGG16 model
        # In PyTorch, weights='IMAGENET1k_V1' or 'DEFAULT' is equivalent to weights='imagenet'
        if self.config.params_weights == 'imagenet':
             weights = models.VGG16_Weights.DEFAULT
        else:
             weights = None
             
        self.model = models.vgg16(weights=weights)
        
        # Freeze base layers if include_top is False (Standard logic)
        # In the context of the tutorial: we freeze layers to keep learned features
        for param in self.model.parameters():
            param.requires_grad = False
            
        self.save_model(path=self.config.base_model_path, model=self.model)

    def update_base_model(self):
        # Recreate the classifier head
        # VGG16 classifier structure in PyTorch is self.model.classifier
        # It usually outputs 1000 classes. We need to change it to our number of classes.
        
        # Get the number of inputs to the classifier
        num_features = self.model.classifier[0].in_features
        
        # Define new classifier
        # Keeping it simple: Flatten -> Dense -> Softmax equivalent
        # Since VGG16 features are already flattened before classifier, we just replace the classifier sequential block
        # Tutorial typically adds a simple Dense layer at the end.
        
        self.model.classifier = nn.Sequential(
            nn.Linear(num_features, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, self.config.params_classes)
            # Note: CrossEntropyLoss includes Softmax, so no separate Softmax layer needed for training
        )
        
        # Unfreeze the new classifier layers to allow training
        for param in self.model.classifier.parameters():
            param.requires_grad = True

        self.save_model(path=self.config.updated_base_model_path, model=self.model)
        print(self.model)

    @staticmethod
    def save_model(path: Path, model: nn.Module):
        torch.save(model, path)


In [ ]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e